In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import cv2 as cv

import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, concatenate
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.utils import plot_model, save_img
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

In [2]:
base_dir = '../input'

In [6]:
plant_village_dir = '/content/drive/MyDrive/plantvillage dataset'

In [10]:
fnames = []

for dir in os.listdir(os.path.join(plant_village_dir, 'color')):
    columns = dir.split('___')
    columns.append(dir)
    fnames.append(columns)

In [11]:
fnames = pd.DataFrame(fnames, columns=['Tree type', 'Disease', 'Folder'])
fnames

,Tree type,Disease,Folder
0,Apple,Apple_scab,Apple___Apple_scab
1,Apple,Black_rot,Apple___Black_rot
2,Apple,Cedar_apple_rust,Apple___Cedar_apple_rust
3,Apple,healthy,Apple___healthy
4,Blueberry,healthy,Blueberry___healthy
5,Cherry_(including_sour),Powdery_mildew,Cherry_(including_sour)___Powdery_mildew
6,Cherry_(including_sour),healthy,Cherry_(including_sour)___healthy
7,Corn_(maize),Cercospora_leaf_spot Gray_leaf_spot,Corn_(maize)___Cercospora_leaf_spot Gray_leaf_...
8,Corn_(maize),Common_rust_,Corn_(maize)___Common_rust_
9,Corn_(maize),Northern_Leaf_Blight,Corn_(maize)___Northern_Leaf_Blight


In [12]:
fnames.drop([4,23,24],axis=0,inplace=True)
fnames

,Tree type,Disease,Folder
0,Apple,Apple_scab,Apple___Apple_scab
1,Apple,Black_rot,Apple___Black_rot
2,Apple,Cedar_apple_rust,Apple___Cedar_apple_rust
3,Apple,healthy,Apple___healthy
5,Cherry_(including_sour),Powdery_mildew,Cherry_(including_sour)___Powdery_mildew
6,Cherry_(including_sour),healthy,Cherry_(including_sour)___healthy
7,Corn_(maize),Cercospora_leaf_spot Gray_leaf_spot,Corn_(maize)___Cercospora_leaf_spot Gray_leaf_...
8,Corn_(maize),Common_rust_,Corn_(maize)___Common_rust_
9,Corn_(maize),Northern_Leaf_Blight,Corn_(maize)___Northern_Leaf_Blight
10,Corn_(maize),healthy,Corn_(maize)___healthy


In [13]:
fnames['Disease'] = fnames['Disease'].str.replace('_', ' ')
fnames['Disease'] = fnames['Disease'].str.lstrip()
fnames['Tree type'] = fnames['Tree type'].str.replace('_', ' ')
fnames['Tree type'] = fnames['Tree type'].str.lstrip()
fnames

,Tree type,Disease,Folder
0,Apple,Apple scab,Apple___Apple_scab
1,Apple,Black rot,Apple___Black_rot
2,Apple,Cedar apple rust,Apple___Cedar_apple_rust
3,Apple,healthy,Apple___healthy
5,Cherry (including sour),Powdery mildew,Cherry_(including_sour)___Powdery_mildew
6,Cherry (including sour),healthy,Cherry_(including_sour)___healthy
7,Corn (maize),Cercospora leaf spot Gray leaf spot,Corn_(maize)___Cercospora_leaf_spot Gray_leaf_...
8,Corn (maize),Common rust,Corn_(maize)___Common_rust_
9,Corn (maize),Northern Leaf Blight,Corn_(maize)___Northern_Leaf_Blight
10,Corn (maize),healthy,Corn_(maize)___healthy


In [14]:
os.mkdir('image data')

In [15]:
os.mkdir(os.path.join('image data', 'train'))
os.mkdir(os.path.join('image data', 'validation'))
os.mkdir(os.path.join('image data', 'test'))

In [16]:
train_path = os.path.join('image data/train')
val_path = os.path.join('image data/validation')
test_path = os.path.join('image data/test')

In [17]:
for tree in fnames['Tree type'].unique().tolist():
    # create directory for tree
    path_1 = os.path.join(train_path, tree)
    path_2 = os.path.join(test_path, tree)
    path_3 = os.path.join(val_path, tree)

    os.mkdir(path_1)
    os.mkdir(path_2)
    os.mkdir(path_3)

    for disease, folder in fnames[fnames['Tree type'] == tree][['Disease', 'Folder']].values.tolist():
        # create directory to each disease
        sub_path_1 = os.path.join(path_1, disease.strip())
        sub_path_2 = os.path.join(path_2, disease.strip())
        sub_path_3 = os.path.join(path_3, disease.strip())

        os.mkdir(sub_path_1)
        os.mkdir(sub_path_2)
        os.mkdir(sub_path_3)

        # read data from source folder
        src = os.path.join(plant_village_dir, 'color', folder)
        image_count = len(os.listdir(src))
        test_img_count = image_count // 10
        val_img_count = (image_count - test_img_count) // 5
        train_img_count = image_count - (test_img_count + val_img_count)
        counts = [train_img_count, test_img_count, val_img_count]
        splits = [sub_path_1, sub_path_2, sub_path_3]

        for i in range(3):
            images = os.listdir(src)
            for j in images[:counts[i]]:
                shutil.copyfile(src=os.path.join(src, j),
                                dst=os.path.join(splits[i], j))

In [18]:
X = []
y = []
val_x = []
val_y = []
test_x = []
test_y = []
tree_types = os.listdir('image data/train')

for tree in tree_types:
    tree_path = os.path.join('image data/train', tree)
    tree_disease_types = os.listdir(tree_path)

    for disease in tree_disease_types:
        img_name = os.listdir(os.path.join(tree_path, disease))

        for img in img_name:
            image_path = os.path.join(tree_path, disease, img)
            image = img_to_array(load_img(image_path, target_size=(56,56)))
            X.append(image)
            y.append([tree, disease])
            
for tree in tree_types:
    tree_path = os.path.join('image data/validation', tree)
    tree_disease_types = os.listdir(tree_path)

    for disease in tree_disease_types:
        img_name = os.listdir(os.path.join(tree_path, disease))

        for img in img_name:
            image_path = os.path.join(tree_path, disease, img)
            image = img_to_array(load_img(image_path, target_size=(56,56)))
            val_x.append(image)
            val_y.append([tree, disease])
            
for tree in tree_types:
    tree_path = os.path.join('image data/test', tree)
    tree_disease_types = os.listdir(tree_path)

    for disease in tree_disease_types:
        img_name = os.listdir(os.path.join(tree_path, disease))

        for img in img_name:
            image_path = os.path.join(tree_path, disease, img)
            image = img_to_array(load_img(image_path, target_size=(56,56)))
            test_x.append(image)
            test_y.append([tree, disease])

In [19]:
mlb = MultiLabelBinarizer()

X = np.array(X)
val_x = np.array(val_x)
y = mlb.fit_transform(y)
val_y = mlb.transform(val_y)
test_y = mlb.transform(test_y)
test_x = np.array(test_x)/255
val_x = val_x/255
X.shape, y.shape

((34112, 56, 56, 3), (34112, 32))

In [20]:
generator = ImageDataGenerator(rescale=1. / 255,
                               rotation_range=15,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               horizontal_flip=True,
                               vertical_flip=True
                                    )
generator.fit(X)

In [21]:
check_pointer = ModelCheckpoint(filepath='exp_conv_20.hdf5',
                                save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss',
                           patience=40,
                           min_delta=0,
                           restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=20,
                             cooldown=15)

In [22]:
def inception_module(x, filter_1x1, filter_3x3_reduce, filter_3x3, filter_5x5_reduce, filter_5x5, filters_pool_proj,
                     name=None):
    # reduction layer
    conv_3x3_reducer = Conv2D(filters=filter_3x3_reduce, kernel_size=(1, 1), activation=relu, padding='same')(x)
    conv_5x5_reducer = Conv2D(filters=filter_5x5_reduce, kernel_size=(1, 1), activation=relu, padding='same')(x)
    pool_3x3 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)

    # extraction layer
    conv_3x3 = Conv2D(filters=filter_3x3, kernel_size=(3, 3), activation=relu, padding='same')(conv_3x3_reducer)
    conv_5x5 = Conv2D(filters=filter_5x5, kernel_size=(5, 5), activation=relu, padding='same')(conv_5x5_reducer)
    conv_1x1_proj = Conv2D(filters=filters_pool_proj, kernel_size=(1, 1), activation=relu, padding='same')(pool_3x3)

    # projection layer
    proj = Conv2D(filters=filter_1x1, kernel_size=(1, 1), activation=relu, padding='same')(x)

    # output
    x = concatenate([proj, conv_1x1_proj, conv_3x3, conv_5x5], axis=3, name=name)

    return x

In [43]:
exp_input = Input(shape=(56,56,3))

# conv_7x7
x = Conv2D(filters=64, kernel_size=(7,7), padding='same', activation=relu)(exp_input)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(1,1))(x)

# conv_5x5
x = Conv2D(filters=192, kernel_size=(5,5), padding='same', activation=relu)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(1,1))(x)

# inception block 1
x = inception_module(x,
                     filter_1x1=64,
                     filter_5x5_reduce=96,
                     filter_3x3=128,
                     filter_3x3_reduce=16,
                     filter_5x5=32,
                     filters_pool_proj=32,
                     name='inception_1a')
x = inception_module(x,
                     filter_1x1=96,
                     filter_5x5_reduce=128,
                     filter_3x3=160,
                     filter_3x3_reduce=32,
                     filter_5x5=48,
                     filters_pool_proj=64,
                     name='inception_1b')

# pooling
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2), padding='same')(x)

# inception block 2
x = inception_module(x,
                     filter_1x1=128,
                     filter_5x5_reduce=96,
                     filter_3x3=128,
                     filter_3x3_reduce=48,
                     filter_5x5=32,
                     filters_pool_proj=32,
                     name='inception_2a')
x = inception_module(x,
                     filter_1x1=160,
                     filter_5x5_reduce=112,
                     filter_3x3=160,
                     filter_3x3_reduce=32,
                     filter_5x5=64,
                     filters_pool_proj=64,
                     name='inception_2b')
x = inception_module(x,
                     filter_1x1=192,
                     filter_5x5_reduce=128,
                     filter_3x3=128,
                     filter_3x3_reduce=48,
                     filter_5x5=64,
                     filters_pool_proj=64,
                     name='inception_2c')
x = inception_module(x,
                     filter_1x1=256,
                     filter_5x5_reduce=160,
                     filter_3x3=160,
                     filter_3x3_reduce=32,
                     filter_5x5=96,
                     filters_pool_proj=96,
                     name='inception_2d')

# pooling
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2), padding='same')(x)

# conv 3x3
x = Conv2D(filters=256, kernel_size=(3,3), activation=relu, padding='same')(x)
x = Conv2D(filters=256, kernel_size=(3,3), activation=relu, padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2), padding='same')(x)

x = Conv2D(filters=256, kernel_size=(3,3), activation=relu, padding='same')(x)
x = Conv2D(filters=256, kernel_size=(3,3), activation=relu, padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2), padding='same')(x)

x = Conv2D(filters=256, kernel_size=(3,3), activation=relu, padding='same')(x)
x = Conv2D(filters=256, kernel_size=(3,3), activation=relu, padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2), padding='same')(x)

# flatten
x = Flatten()(x)

# output layer
x = Dense(units=1024, activation=relu)(x)
x = Dropout(0.5)(x)
x = Dense(units=1024, activation=relu)(x)
x = Dropout(0.5)(x)
exp_output = Dense(units=32, activation=sigmoid)(x)     # units 개수?????

exp_conv = Model(exp_input, exp_output)
exp_conv.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 56, 56, 3)]  0           []                               
                                                                                                  
 conv2d_264 (Conv2D)            (None, 56, 56, 64)   9472        ['input_7[0][0]']                
                                                                                                  
 batch_normalization_42 (BatchN  (None, 56, 56, 64)  256         ['conv2d_264[0][0]']             
 ormalization)                                                                                    
                                                                                                  
 max_pooling2d_78 (MaxPooling2D  (None, 54, 54, 64)  0           ['batch_normalization_42[0]

In [44]:
# plot_model(exp_conv,
#            to_file='baseline_conv_15_exp(VGGNet with INCEPTION).png',
#            show_shapes=True,
#            show_dtype=True,
#            show_layer_names=True)

In [ ]:
exp_conv.compile(optimizer=Adam(),
                   loss=binary_crossentropy,
                   metrics=[Recall(), Precision(), 'accuracy'])

history = exp_conv.fit(generator.flow(X,y, batch_size=128), 
                        epochs=500, 
                        validation_data=(val_x,val_y),
                        callbacks=[early_stop, check_pointer, reduce_lr])

Epoch 1/500
267/267 [==============================] - 256s 909ms/step - loss: 0.1574 - recall_8: 0.3370 - precision_8: 0.6625 - accuracy: 0.2392 - val_loss: 0.2872 - val_recall_8: 0.1332 - val_precision_8: 0.1807 - val_accuracy: 0.1119 - lr: 0.0010
Epoch 2/500
267/267 [==============================] - 233s 873ms/step - loss: 0.1011 - recall_8: 0.5579 - precision_8: 0.8078 - accuracy: 0.3368 - val_loss: 0.1608 - val_recall_8: 0.4183 - val_precision_8: 0.5558 - val_accuracy: 0.2432 - lr: 0.0010
Epoch 3/500
267/267 [==============================] - 232s 869ms/step - loss: 0.0762 - recall_8: 0.6833 - precision_8: 0.8425 - accuracy: 0.3712 - val_loss: 0.1649 - val_recall_8: 0.4155 - val_precision_8: 0.5861 - val_accuracy: 0.2953 - lr: 0.0010
Epoch 4/500
267/267 [==============================] - 232s 869ms/step - loss: 0.0584 - recall_8: 0.7680 - precision_8: 0.8744 - accuracy: 0.3975 - val_loss: 0.1695 - val_recall_8: 0.5205 - val_precision_8: 0.5905 - val_accuracy: 0.2977 - lr: 0.0010


In [ ]:
exp_conv.evaluate(test_x,test_y)

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=3, figsize=[18, 6], dpi=300)
axes = axes.ravel()
epochs = list(range(len(history.history['loss'])))

sns.lineplot(x=epochs, y=history.history['loss'], ax=axes[0],label='loss')
sns.lineplot(x=epochs, y=history.history['val_loss'], ax=axes[0],label='val loss')
sns.lineplot(x=epochs, y=history.history['precision'], ax=axes[1],label='precision')
sns.lineplot(x=epochs, y=history.history['val_precision'], ax=axes[1],label='val precision')
sns.lineplot(x=epochs, y=history.history['recall'], color='#025918', ax=axes[1],label='recall')
sns.lineplot(x=epochs, y=history.history['val_recall'], color='#D9B504',ax=axes[1],label='val recall')
sns.lineplot(x=epochs, y=history.history['accuracy'], ax=axes[2],label='accuracy')
sns.lineplot(x=epochs, y=history.history['val_accuracy'], ax=axes[2],label='val accuracy')
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('loss')
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('precision and recall')
axes[2].set_xlabel('epoch')
axes[2].set_ylabel('accuracy')
plt.savefig('VGGNet_base_conv_train_history_with_cassava.png')
plt.show()

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=2, figsize=[12, 6], dpi=300)
axes = axes.ravel()

sns.lineplot(x=epochs, y=history.history['lr'], ax=axes[0],label='learning rate')
sns.lineplot(x=history.history['lr'], y=history.history['val_accuracy'], ax=axes[1],label='accuracy & lr')
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('learning rate')
axes[1].set_xlabel('learning rate')
axes[1].set_ylabel('accuracy')

plt.savefig('VGGNet_base_conv_lr_history_4.png')
plt.show()

In [ ]:
# Plot confusion matrix
fig, axes = plt.subplots(nrows=14, ncols=3, figsize=[24, 72], dpi=300)
axes = axes.ravel()

preds = np.where(exp_conv.predict(test_x) < 0.65, 0, 1)
confusion = multilabel_confusion_matrix(test_y, preds)

for i, (label, matrix) in enumerate(zip(mlb.classes_, confusion)):
    labels = [f'not_{label}', label]
    sns.heatmap(matrix, annot=True, square=True, fmt='d', cbar=False, cmap='Blues',
                xticklabels=labels, yticklabels=labels, linecolor='black', linewidth=1,
                ax=axes[i])
    axes[i].set_title(labels[0])

plt.savefig('VGGNet_base_conv_4_cm.png')
plt.show()

In [ ]:
exp_conv.save('/content/drive/MyDrive/VGGNet_based_with_all_data_be_top.hdf5')

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=2,figsize = (8, 24))
preds = exp_conv.predict(test_x[200:210])
axes = axes.ravel()

for i in range(len(preds)):
    p = zip(list(mlb.classes_), list(preds[i]))
    p = sorted(list(p), key = lambda z: z[1], reverse = True)[:2]
    axes[i].imshow(test_x[200+i])
    axes[i].set_title(f'{p[0][0]}: {round(p[0][1] * 100, 2)}% \n {p[1][0]}: {round(p[1][1] * 100, 2)}%')

In [ ]:
pd.to_pickle(pd.DataFrame(history.history), 'VGGNet_base_conv_lr_history.pkl')